In [1]:
import sys, os

ppath = lambda x: os.path.dirname(os.path.abspath(x))
file_name = os.getcwd()
sys.path.append(ppath(ppath(file_name)))

from recorder.recorder import Recorder

In [2]:
import gym

from ppo import *
from torch.distributions import Categorical

In [3]:
# conda install torchvision
# conda install -c conda-forge tensorboard

from torch.utils.tensorboard import SummaryWriter

In [4]:
def main():
    recorder = Recorder(gym.make('CartPole-v1'), False)
    env = recorder.wrapped_env()
    writer = SummaryWriter()

    model = PPO()
    score = 0.0
    print_interval = 20

    for n_epi in range(10000):
        s = env.reset()
        done = False
        while not done:
            for t in range(T_horizon):
                prob = model.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                
                # r은 크기가 너무 커서 1/100을 했다. 그래야 학습이 잘 됨.
                # prob[a].item()은 a를 할 확률을 말함
                model.put_data((s, a, r/100.0, s_prime, prob[a].item(), done))
                s = s_prime

                score += r
                if done:
                    break

            model.train_net()
        
        if score / print_interval > 490:
            recorder.update([n_epi + 1])
            if (len(recorder.recorded_epi()) > 5):
                env.reset()
                break

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode: {}, avg score: {:.1f}".format(n_epi, score/print_interval))
            writer.add_scalar("score/train", score/print_interval, n_epi)
            score = 0.0

    env.close()
    writer.flush()
    writer.close()

In [5]:
if __name__ == '__main__':
    main()

# of episode: 20, avg score: 28.8
# of episode: 40, avg score: 36.2
# of episode: 60, avg score: 54.9
# of episode: 80, avg score: 49.2
# of episode: 100, avg score: 111.3
# of episode: 120, avg score: 49.7
# of episode: 140, avg score: 78.0
# of episode: 160, avg score: 138.9
# of episode: 180, avg score: 255.4
# of episode: 200, avg score: 190.4
# of episode: 220, avg score: 134.1
# of episode: 240, avg score: 115.5
# of episode: 260, avg score: 158.3
# of episode: 280, avg score: 146.1
# of episode: 300, avg score: 170.9
# of episode: 320, avg score: 213.1
# of episode: 340, avg score: 157.4
# of episode: 360, avg score: 224.1
# of episode: 380, avg score: 323.6
# of episode: 400, avg score: 124.9
# of episode: 420, avg score: 175.2
# of episode: 440, avg score: 356.4
# of episode: 460, avg score: 486.4
# of episode: 480, avg score: 433.6
# of episode: 500, avg score: 441.0
# of episode: 520, avg score: 402.8
# of episode: 540, avg score: 338.0
# of episode: 560, avg score: 166.2
# 

C:\Users\kuro1\anaconda3\envs\py38-pytorch-gpu\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] 스레드 모드가 설정된 후에는 바꿀 수 없습니다
  warnings.warn(str(err))


# of episode: 1180, avg score: 500.0
# of episode: 1200, avg score: 500.0
# of episode: 1220, avg score: 496.4
# of episode: 1240, avg score: 473.3
# of episode: 1260, avg score: 476.0
# of episode: 1280, avg score: 500.0
# of episode: 1300, avg score: 489.5
# of episode: 1320, avg score: 411.9
# of episode: 1340, avg score: 398.8
# of episode: 1360, avg score: 262.9
# of episode: 1380, avg score: 126.8
# of episode: 1400, avg score: 128.9
# of episode: 1420, avg score: 147.8
# of episode: 1440, avg score: 139.4
# of episode: 1460, avg score: 94.0
# of episode: 1480, avg score: 173.4
# of episode: 1500, avg score: 435.4


In [6]:
home_path = os.path.expanduser('~')
cur_path = os.getcwd()
conda_path = home_path + "\\anaconda3"
conda_script_path = home_path + "\\anaconda3\\Scripts\\activate.bat"
exc = ' '.join(['start', '%windir%\System32\cmd.exe "/K"', conda_script_path, conda_path])
!$exc

conda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs

http://localhost:6006/